In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12505697535180575894
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1507328
locality {
  bus_id: 1
}
incarnation: 16988791617578994689
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1"
]


In [23]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import time

import scipy.io
import numpy as np
import tensorflow as tf
import h5py
import random
import matplotlib.pyplot as plt
import os
import cv2
import itertools
from tensorflow.contrib import layers
from tensorflow.python.layers.core import Dense

%matplotlib inline

In [ ]:
#Input
def load_Images(Data_Train_Path):
    main_path = [f for f in np.sort(os.listdir(Data_Train_Path)) if f.endswith('redraw')]
    height = 64
    imgs = []
    for s in main_path:
        sub = Data_Train_Path + s
        sub_path = [f for f in np.sort(os.listdir(sub)) if f.endswith('.tif')]
        for t in sub_path:
            im_path = sub + '/' + t           
            img = cv2.imread(im_path,cv2.IMREAD_UNCHANGED)
            imgs.append(img)            
    return np.expand_dims(imgs,-1)

In [ ]:
#Input
def load_Coordinates(Data_Train_Path):
    Data_Label = []
    DATA_Label_K = []
    main_path = [f for f in np.sort(os.listdir(Data_Train_Path)) if f.endswith('online')]
    for s in main_path:
        sub = Data_Train_Path + s
        sub_path = [f for f in np.sort(os.listdir(sub)) if f.endswith('.txt')]
        for t in sub_path:
            txt_path = sub + '/' + t            
            file_B = open(txt_path, 'r') 
            data = file_B.read()
            x = data.split('\n')
            online_data = np.zeros([50,2],np.int32)           
            online_data_K = np.zeros([51,2],np.int32)
            
            for y in range(50):
                z = x[y].split(" ")
                online_data[y] = [int(z[0]),int(z[1])]
                online_data_K[y+1] = [int(z[0]),int(z[1])]
            online_data_K = online_data_K[:50,:]
            #print(online_data)
            file_B.close() 
            Data_Label.append(online_data)
            DATA_Label_K.append(online_data_K)
    return np.array(Data_Label), np.array(DATA_Label_K)

In [ ]:
def Architecture(Input, outputs, is_training = True):
    #Input,_,_ = load_patch_data(Data_Train_Path,height = 224,win_size = 224,win_step = 112,return_all = True)
    #Input.shape           
    with tf.variable_scope("Network"):
        
        with tf.variable_scope("CNN"):
            conv1 = tf.layers.conv2d(Input,64,3,activation=tf.nn.relu,name = 'conv1', padding = 'same')
            pool1 = tf.layers.max_pooling2d(conv1,2,[2,2],name = 'pool1')

            conv2 = tf.layers.conv2d(pool1,128,3,activation=tf.nn.relu,name = 'conv2', padding = 'same')
            pool2 = tf.layers.max_pooling2d(conv2, 2,[2,1],name = 'pool2',  padding = 'same')


            conv3 = tf.layers.conv2d(pool2,256,3,activation=None,name = 'conv3', padding = 'same')
            b_norm3 = tf.nn.relu(tf.layers.batch_normalization(conv3, training = is_training, name='batch-norm1'), name = 'relu3')


            conv4 = tf.layers.conv2d(b_norm3,256,3,activation=tf.nn.relu,name = 'conv4', padding = 'same')
            pool4 = tf.layers.max_pooling2d(conv4, 2,[2,1],name = 'pool4',  padding = 'same')

            conv5 = tf.layers.conv2d(pool4,256,3,activation=tf.nn.relu,name = 'conv5', padding = 'same')
            pool5 = tf.layers.max_pooling2d(conv5, 2,[2,1],name = 'pool5',  padding = 'same')

            conv6 = tf.layers.conv2d(pool5,256,3,activation=None,name = 'conv6', padding = 'same')
            b_norm6 = tf.nn.relu(tf.layers.batch_normalization(conv6, training = is_training, name='batch-norm2'), name = 'relu6')

            conv7 = tf.layers.conv2d(b_norm6,256,3,activation=tf.nn.relu,name = 'conv7', padding = 'same')
            pool7 = tf.layers.max_pooling2d(conv7, 2,[2,1],name = 'pool7',  padding = 'same')

            conv8 = tf.layers.conv2d(pool7,256,2,activation=None, name = 'conv8')
            b_norm8 = tf.nn.relu(tf.layers.batch_normalization(conv8, training = is_training, name='batch-norm3'), name = 'relu8')

            shape = b_norm8.get_shape().as_list()
            transposed = tf.transpose(b_norm8, perm=[0, 2, 1, 3], name='transposed') 
            conv_reshaped = tf.reshape(transposed, [ shape[0], -1, shape[1] * shape[3] ], name='reshaped') 
            
        num_units = 256
        num_layers = 3
        
        cell_stack_encoder = []
        cell_stack_decoder = []

        with tf.variable_scope("Encoder"):
            for i in range(num_layers):
                cell_stack_encoder.append(tf.contrib.rnn.LSTMCell(num_units))        
            encoder_cell = tf.contrib.rnn.MultiRNNCell(cell_stack_encoder, state_is_tuple=True)
            _, encoder_state = tf.nn.dynamic_rnn(encoder_cell, conv_reshaped, dtype = tf.float32)

        with tf.variable_scope("Decoder"):
            for i in range(num_layers):
                cell_stack_decoder.append(tf.contrib.rnn.LSTMCell(num_units))  
            decoder_cell = tf.contrib.rnn.MultiRNNCell(cell_stack_decoder, state_is_tuple=True)
            dec_outputs, _ = tf.nn.dynamic_rnn(decoder_cell, outputs, initial_state = encoder_state, dtype = tf.float32)    
            
            logits = tf.layers.dense(dec_outputs, 2, name  = 'logits')  
        
            return logits    

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
Input = tf.placeholder(tf.float32, shape = [BatchSize, 64, 64, 1])
outputs = tf.placeholder(tf.float32, shape = [BatchSize, 64, 2])
Architecture(Input, outputs)

In [ ]:
def Loss_function(Dec_Outputs, Targets):
    loss = tf.reduce_mean(tf.abs(Dec_Outputs - Targets), name = 'loss')
    return loss

In [ ]:
def Loss_function_Test(Dec_Outputs, Targets):
    loss_ = np.sum(np.abs(Dec_Outputs - Targets))
    return loss_

In [ ]:
def reDraw(dataTensor,predTensor, num, isTrain):
    for x in range(num):        
        imgData = np.zeros([64,64],np.int32)
        imgPred = np.zeros([64,64],np.int32)

        for y in range(50):        
            if (dataTensor[x,y,0] < 64) and (dataTensor[x,y,1] < 64):
                imgData[dataTensor[x,y,1],dataTensor[x,y,0]] = 1
            if (predTensor[x,y,0] < 64) and (predTensor[x,y,1] < 64):
                imgPred[predTensor[x,y,1],predTensor[x,y,0]] = 1

        fig = plt.figure(figsize = (12,12))
        fig.add_subplot(1,2,1)
        plt.imshow(imgData)
        plt.title('Ground Truth' + isTrain)

        fig.add_subplot(1,2,2)
        plt.imshow(imgPred)
        plt.title('Predicted' + isTrain)   

    #plt.tight_layout()    
    plt.show()

In [ ]:
print("Setting up Saver...")
saver = tf.train.Saver()
summary_writer = tf.summary.FileWriter('./logs_dir_TELEGU_Error/', sess.graph)
ckpt = tf.train.get_checkpoint_state('./logs_dir_TELEGU_Error/')


if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Model restored...")

In [ ]:
def Compile(Data_Train,LabelTrain_Outputs, LabelTrain_Targets, BatchSize = 32, learning_rate = 0.001, num_epochs = 50):

    #Data_Train,Train_Label,Train_seq = load_patch_data(params['trainpath'])
    #Data_Test,Test_Label,Test_Seq = load_patch_data(params['valpath'])
    [Total_Data, row, col, depth] = np.shape(Data_Train)
    
    
    random_index = np.random.choice(Total_Data, 1000, replace=False)
    
    Data_Test = Data_Train[random_index,:,:,:]
    LabelTest_Outputs = LabelTrain_Outputs[random_index, :,:]
    LabelTest_Targets = LabelTrain_Targets[random_index, :,:]
    
    
    Data_Train = np.delete(Data_Train,random_index, axis = 0)
    LabelTrain_Outputs = np.delete(LabelTrain_Outputs,random_index, axis = 0)
    LabelTrain_Targets = np.delete(LabelTrain_Targets,random_index, axis = 0)
    
    [Total_Data, row, col, depth] = np.shape(Data_Train)
    

    
    
    

    Inputs = tf.placeholder(tf.float32, shape = [BatchSize, 64, 64, 1], name = 'Inputs')
    outputs = tf.placeholder(tf.float32, shape = [BatchSize, None, 2], name = 'outputs')
    targets = tf.placeholder(tf.float32, shape = [BatchSize, 50, 2], name = 'targets')


    logits = Architecture(Inputs, outputs)
    loss = Loss_function(logits, targets)
    
     

    training_step = tf.train.AdamOptimizer(learning_rate = 0.001, name='Adam').minimize(loss)


    init = tf.global_variables_initializer()
    sess.run(init)

    train_loss_list = []
    mean_loss_list = []

    timeperepoch = []

    
    tf.summary.scalar("TrainLoss", loss)
    tf.summary.histogram("X_Value", logits[:,0])
    tf.summary.histogram("Y_Value", logits[:,1])
    
    
    
    
    #tf.summary.image("Target", draw_Targ, max_outputs = 5)
    #tf.summary.image("Pred", draw_Pred, max_outputs = 5)
    
    
    print("Setting up summary op...")
    summary_op = tf.summary.merge_all()


    print("Setting up Saver...")
    saver = tf.train.Saver()
    summary_writer = tf.summary.FileWriter('./logs_dir_TELEGU_Error/', sess.graph)
    ckpt = tf.train.get_checkpoint_state('./logs_dir_TELEGU_Error/')
    
    
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print("Model restored...")
    
    itr = 0
    
    for i in range(num_epochs):

        index = np.random.permutation(Total_Data)
        
        Data_Train = Data_Train[index,:,:,:]
        
        LabelTrain_Targets = LabelTrain_Targets[index,:,:]
        LabelTrain_Outputs = LabelTrain_Outputs[index,:,:]
        
        for k in range(Total_Data//BatchSize):         
            start = time.time()
            
            batch_input = Data_Train[k*BatchSize:(k+1)*BatchSize,:,:,:]                      
            batch_outputs = LabelTrain_Outputs[k*BatchSize:(k+1)*BatchSize,:,:]
            batch_targets = LabelTrain_Targets[k*BatchSize:(k+1)*BatchSize,:,:] 
            train_feed_dict = {Inputs:batch_input, outputs:batch_outputs, targets:batch_targets}
            
            
            _,train_loss,summary_str,logits_Out = sess.run([training_step,loss,summary_op,logits],train_feed_dict)
            summary_writer.add_summary(summary_str, itr)
            itr = itr + 1
            
            train_loss_list.append(train_loss)
           
            end = time.time()    
             
            timeperepoch.append(end-start)

            #print('Epoch: '+str(i) + ' Step:' + str(k) + ' train loss :'+ str(train_loss) + '  Time per epoch : ' +str(timeperepoch[-1]) )
            
            
            if k%300 == 0:
                
                index = np.random.permutation(1000)
                Data_Test = Data_Test[index,:,:,:]     
                LabelTest_Targets = LabelTest_Targets[index,:,:]
                
                
                
                Test_loss = []
                BatchSize_Test = BatchSize
                for jk in range(5): #(1000//BatchSize_Test):                             
                    batch_Test_input = Data_Test[jk*BatchSize_Test:(jk+1)*BatchSize_Test,:,:,:]  
                    batch_Test_targets = LabelTest_Targets[jk*BatchSize_Test:(jk+1)*BatchSize_Test,:,:]                         
                    batch_Test_outputs =  np.zeros(shape = [BatchSize_Test, 1, 2])

                    for ij in range(50):
                        test_feed_dict = {Inputs:batch_Test_input, outputs:batch_Test_outputs}
                        logits_Out = sess.run([logits], feed_dict = test_feed_dict)
                        logits_Out = logits_Out[0]
                        #print(logits_Out.shape)
                        prediction = np.expand_dims(logits_Out[:,-1,:],axis = 1)
                        batch_Test_outputs = np.hstack([batch_Test_outputs, prediction])
                        
                    loss_ = Loss_function_Test(logits_Out, batch_Test_targets) 

                    Test_loss.append(loss_)
                    if jk%3 == 0:
                        reDraw(batch_Test_targets.astype(int), logits_Out.astype(int),2, ' Testing')
                
                    
                Test_loss = np.mean(Test_loss)
                print('### TESTING TESTING TESING ###')
                print('Epoch: '+str(i) + ' Step:' + str(k) + 'Test Loss is: => ' + str(Test_loss) + '\n')
                print('### TESTING TESTING TESING ###')
                
                
            #if k%100 == 0:
            #    reDraw(batch_targets.astype(int),logits_Out.astype(int), 3, ' Training')     
            

        
        
        if i%10 == 0:
            saver.save(sess, './logs_dir_TELEGU_Error/' + "model.ckpt", i)  
               
    return random_index

In [ ]:
LabelTrain_Targets, LabelTrain_Outputs = load_Coordinates('/media/ayan/Drive/Offline2Online/Tamil/telugu/')
DataTrain = load_Images('/media/ayan/Drive/Offline2Online/Tamil/telugu/')

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

global sess

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session (config=config)
graph = tf.get_default_graph()

In [ ]:
random_index = Compile(DataTrain,LabelTrain_Outputs, LabelTrain_Targets)

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

Input = tf.placeholder(tf.float32, shape = [32, 64, 64, 1], name = 'Inputs')
with tf.variable_scope("Network"):
    with tf.variable_scope("CNN"):
        conv1 = tf.layers.conv2d(Input,64,3,activation=tf.nn.relu,name = 'conv1', padding = 'same')
        pool1 = tf.layers.max_pooling2d(conv1,2,[2,2],name = 'pool1')
            
ckpt = tf.train.get_checkpoint_state('./logs_dir_TELEGU_Error/')
saver = tf.train.Saver()
sess = tf.Session()
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Model restored...")
#print(sess.run(tf.trainable_variables()[0]))

In [9]:
tf.trainable_variables()

[]

In [ ]:
Data_Test = DataTrain[random_index,:,:,:]
LabelTest_Outputs = LabelTrain_Outputs[random_index, :,:]
LabelTest_Targets = LabelTrain_Targets[random_index, :,:]


Data_Train = np.delete(DataTrain,random_index, axis = 0)
LabelTrain_Outputs = np.delete(LabelTrain_Outputs,random_index, axis = 0)
LabelTrain_Targets = np.delete(LabelTrain_Targets,random_index, axis = 0)

#[Total_Data, row, col, depth] = np.shape(Data_Train)

In [24]:
def get_graph(model_dir = './logs_dir_TELEGU_Error/'): 
    ckpt_file_path  = model_dir + [i for i in  os.listdir(model_dir) if i.endswith('meta')][0]
    loader = tf.train.import_meta_graph(ckpt_file_path)
    loader.restore(sess, tf.train.latest_checkpoint(model_dir))
    graph = tf.get_default_graph()
    return graph

In [29]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

global graph
global sess
sess = tf.Session ()
graph= get_graph()

INFO:tensorflow:Restoring parameters from ./logs_dir_TELEGU_Error/model.ckpt-40


In [8]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:
#LabelTrain_Targets, LabelTrain_Outputs = load_Coordinates('/media/ayan/Drive/Offline2Online/Tamil/telugu/')
#Data_Train = load_Images('/media/ayan/Drive/Offline2Online/Tamil/telugu/')

#[Total_Data, row, col, depth] = np.shape(Data_Train)


#random_index = np.random.choice(Total_Data, 1000, replace=False)

#Data_Test = Data_Train[random_index,:,:,:]
#LabelTest_Outputs = LabelTrain_Outputs[random_index, :,:]
#LabelTest_Targets = LabelTrain_Targets[random_index, :,:]


#Data_Train = np.delete(Data_Train,random_index, axis = 0)
#LabelTrain_Outputs = np.delete(LabelTrain_Outputs,random_index, axis = 0)
#LabelTrain_Targets = np.delete(LabelTrain_Targets,random_index, axis = 0)

In [ ]:
Inputs = graph.get_tensor_by_name('Inputs:0')
outputs = graph.get_tensor_by_name('outputs:0')
targets = graph.get_tensor_by_name('targets:0')
logits = graph.get_tensor_by_name('Network/Decoder/logits/BiasAdd:0')

In [30]:
tf.trainable_variables()
kernel_0 = graph.get_tensor_by_name('Network/CNN/conv1/kernel:0')

#init = tf.global_variables_initializer()
#print(kernel_0)

In [31]:
BB = sess.run(kernel_0)
print(BB)

[[[[  3.09421346e-02  -3.80455144e-02   4.18581516e-02   7.33650476e-02
     -4.45071831e-02  -5.49664684e-02  -1.36233032e-01   2.75193043e-02
      1.61895797e-01  -8.27366672e-03  -1.78561747e-01  -3.49660479e-02
      2.03686059e-02  -2.75989976e-02   1.18626125e-01  -9.90138799e-02
     -3.68017815e-02   3.15786637e-02   3.83046381e-02  -5.38569223e-03
      2.30972618e-02   1.40962191e-02   1.11561634e-01   1.05203629e-01
      1.00139953e-01  -2.38342136e-02  -1.96031108e-01  -4.25074110e-03
     -9.76578891e-02   1.13854064e-02  -1.44755065e-01   1.36243328e-01
      1.49112895e-01   3.13988999e-02  -1.51007855e-02   1.36923641e-01
      7.94806853e-02  -1.67758271e-01   1.32514611e-01  -4.27277982e-02
     -4.01914120e-02  -5.22282422e-02  -1.72426164e-01   1.10759810e-01
     -1.23551287e-01   3.88559997e-02  -7.62599707e-02  -1.35854140e-01
      1.92850366e-01   1.89058259e-02   3.57168466e-02  -4.86660190e-02
      9.40493867e-02  -3.91798234e-03  -1.36152387e-01   1.86873

In [16]:
Input = tf.placeholder(tf.float32, shape = [32, 64, 64, 1], name = 'Inputs')
with tf.variable_scope("Network"):
    with tf.variable_scope("CNN"):
        k_init = tf.constant_initializer(BB)
        conv1 = tf.layers.conv2d(Input,64,3,activation=tf.nn.relu,name = 'conv1', kernel_initializer = k_init, padding = 'same')
        pool1 = tf.layers.max_pooling2d(conv1,2,[2,2],name = 'pool1')

In [17]:
tf.trainable_variables()

[<tf.Variable 'Network/CNN/conv1/kernel:0' shape=(3, 3, 1, 64) dtype=float32_ref>,
 <tf.Variable 'Network/CNN/conv1/bias:0' shape=(64,) dtype=float32_ref>]

In [21]:
sess.run(tf.global_variables_initializer())
CC = sess.run(tf.trainable_variables()[0])
BB == CC

array([[[[ True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True]],

        [[ True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True, 

In [ ]:
tf.layers.conv2d(kernel_initializer = )

In [ ]:
plt.imshow(batch_Test_input[0,:,:,0])
plt.show()
BB = batch_Test_input[0,:,:,0]
BB.shape

In [ ]:
np.min(BB)

In [ ]:
fileid = open('testfile.txt','w') 
fileid.write(batch_Test_targets[0,:,:])
fileid.close()

In [ ]:
batch_Test_targets[0,:,:]
np.savetxt('testfile2.txt', batch_Test_targets[0,:,:])

In [ ]:
Test_loss = []
BatchSize = 32
BatchSize_Test = BatchSize
Save_folder = '/media/ayan/Drive/Offline2Online/Final_results_TELEGU_Error/'

number = 0
for jk in range(1000//BatchSize_Test):    
    
    batch_Test_input = Data_Test[jk*BatchSize_Test:(jk+1)*BatchSize_Test,:,:,:]  
    batch_Test_targets = LabelTest_Targets[jk*BatchSize_Test:(jk+1)*BatchSize_Test,:,:]                         
    batch_Test_outputs =  np.zeros(shape = [BatchSize_Test, 1, 2])
    
    for ij in range(50):
        test_feed_dict = {Inputs:batch_Test_input, outputs:batch_Test_outputs}
        logits_Out = sess.run([logits], feed_dict = test_feed_dict)
        logits_Out = logits_Out[0]
        #print(logits_Out.shape)
        prediction = np.expand_dims(logits_Out[:,-1,:],axis = 1)
        batch_Test_outputs = np.hstack([batch_Test_outputs, prediction])

    loss_ = Loss_function_Test(logits_Out, batch_Test_targets) 

    Test_loss.append(loss_)
    
    for num in range(32):
        directory = Save_folder + str(number) + '/'
        try:
            os.stat(directory)
        except:
            os.mkdir(directory) 
        img = 255 - batch_Test_input[num, :,:,0]
        imgpath = directory + str(number) + '.jpg'
        cv2.imwrite(imgpath, img)
        np.savetxt(  directory + 'Grountruth_' + str(num) + '.txt', batch_Test_targets[num,:,:])
        np.savetxt(  directory + 'Predict_' + str(num) + '.txt', logits_Out[num,:,:])
        number = number + 1
        
    if jk%3 == 0:
        reDraw(batch_Test_targets.astype(int), logits_Out.astype(int),2, ' Testing')